In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector

In [3]:
passwd = 'W_a050291'

In [4]:
con = mysql.connector.connect(
    host='localhost',
    user='root',
    passwd=f'{passwd}'
)

In [5]:
con

In [6]:
c = con.cursor(buffered=True)
query = 'SHOW databases'
c.execute(query)

In [7]:
c.fetchall()

[('coffe_store',),
 ('flask_mysql',),
 ('hotwheels',),
 ('information_schema',),
 ('mysql',),
 ('performance_schema',),
 ('sakila',),
 ('sql_hr',),
 ('sql_inventory',),
 ('sql_invoicing',),
 ('sql_store',),
 ('sys',),
 ('world',)]

In [8]:
query = 'USE hotwheels'
c.execute(query)

In [9]:
query = 'SHOW tables'
c.execute(query)
c.fetchall()

[('customers',),
 ('employees',),
 ('offices',),
 ('orderdetails',),
 ('orders',),
 ('payments',),
 ('productlines',),
 ('products',)]

In [10]:
query = 'SELECT * FROM customers'
c.execute(query)
res = c.fetchall()
cols = c.column_names


In [12]:
# bikin DF
customer_df = pd.DataFrame(res, columns = cols)
customer_df.set_index('customerNumber', inplace=True)
customer_df

,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
customerNumber,,,,,,,,,,,,
103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",None,Nantes,None,44000,France,1370.0,21000.00
112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,None,Las Vegas,NV,83030,USA,1166.0,71800.00
114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611.0,117300.00
119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",None,Nantes,None,44000,France,1370.0,118200.00
121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,None,Stavern,None,4110,Norway,1504.0,81700.00
...,...,...,...,...,...,...,...,...,...,...,...,...
486,Motor Mint Distributors Inc.,Salazar,Rosa,2155559857,11328 Douglas Av.,None,Philadelphia,PA,71270,USA,1323.0,72600.00
487,Signal Collectibles Ltd.,Taylor,Sue,4155554312,2793 Furth Circle,None,Brisbane,CA,94217,USA,1165.0,60300.00
489,"Double Decker Gift Stores, Ltd",Smith,Thomas,(171) 555-7555,120 Hanover Sq.,None,London,None,WA1 1DP,UK,1501.0,43300.00


__Pada 2003-06-05, terdapat pembayaran masuk sebesar US$ 14571.44. Tampilkan data seputar transaksi tersebut, mencakup nama customer yang melakukan pembayaran, nama produk yang dibeli, jumlah tiap produk yang dibeli dan harga satuannya. Pastikan total harga yang dibeli sesuai dengan data pembayaran masuk. Output yang diharapkan:__

In [24]:
query_1 = '''select customers.customerName as customerName, products.productName as productName, orderdetails.quantityOrdered as quantityOrdered, orderdetails.priceEach as priceEach, orderdetails.quantityOrdered*orderdetails.priceEach as totalPrice
from customers, products, orderdetails, orders, payments
where customers.customerNumber = payments.customerNumber
and customers.customerNumber = orders.customerNumber
and orders.orderNumber = orderdetails.orderNumber
and orderdetails.productCode = products.productCode
and payments.paymentDate = '2003-06-05'
and orders.orderDate < '2003-06-05'
limit 4;
'''
c.execute(query_1)
c.fetchall()

[('Atelier graphique',
  '1965 Aston Martin DB5',
  26,
  Decimal('120.71'),
  Decimal('3138.46')),
 ('Atelier graphique',
  '1999 Indy 500 Monte Carlo SS',
  46,
  Decimal('114.84'),
  Decimal('5282.64')),
 ('Atelier graphique',
  '1948 Porsche Type 356 Roadster',
  34,
  Decimal('117.26'),
  Decimal('3986.84')),
 ('Atelier graphique',
  '1966 Shelby Cobra 427 S/C',
  50,
  Decimal('43.27'),
  Decimal('2163.50'))]

In [25]:
hasil = c.fetchall()
kolom = c.column_names
tabel1 = pd.DataFrame(hasil, columns = kolom)
tabel1

,customerName,productName,quantityOrdered,priceEach,totalPrice


__2. (poin 5) Tampilkan 10 (sepuluh) orderNumber dengan nominal transaksi terbesar. Tampilkan juga nama customer yang melakukan pemesanan dengan orderNumber terkait. Berikut dengan asal kota dan negaranya. Output yang diharapkan:__

In [26]:
no2 = pd.read_sql("SELECT od.orderNumber, \
c.customerName, \
c.city, \
c.country, \
od.quantityOrdered, \
od.priceEach, \
sum(od.quantityOrdered * od.priceEach) as totalPurchase \
FROM customers c \
JOIN orders o \
USING(customerNumber) \
JOIN orderdetails od \
USING(orderNumber) \
GROUP BY od.orderNumber \
ORDER BY totalPurchase DESC \
LIMIT 10;",con)
no2

,orderNumber,customerName,city,country,quantityOrdered,priceEach,totalPurchase
0,10165,"Dragon Souveniers, Ltd.",Singapore,Singapore,44,168.32,67392.85
1,10287,"Vida Sport, Ltd",Genève,Switzerland,21,190.68,61402.00
2,10310,"Toms Spezialitäten, Ltd",Köln,Germany,33,165.38,61234.67
3,10212,Euro+ Shopping Channel,Madrid,Spain,39,99.82,59830.55
4,10207,Diecast Collectables,Boston,USA,31,125.58,59265.14
5,10127,Muscle Machine Inc,NYC,USA,46,193.25,58841.35
6,10204,Muscle Machine Inc,NYC,USA,42,114.65,58793.53
7,10126,"Corrida Auto Replicas, Ltd",Madrid,Spain,38,205.73,57131.92
8,10222,Collectable Mini Designs Co.,San Diego,USA,49,133.28,56822.65
9,10142,Mini Gifts Distributors Ltd.,San Rafael,USA,33,166.24,56052.56


__3. (poin 5) Tampilkan 10 (sepuluh) mainan yang paling laris (Most Wanted Items) atau yang paling sering diorder oleh customer pada tahun 2003. Output yang diharapkan:__

In [27]:
no3 = pd.read_sql("SELECT p.productName as mostWanted2003, \
sum(od.quantityOrdered) as quantityOrdered \
FROM orders o \
JOIN orderdetails od \
USING(orderNumber) \
JOIN products p \
USING(productCode) \
WHERE o.orderDate REGEXP '^2003' \
GROUP BY mostWanted2003 \
ORDER BY quantityOrdered DESC \
LIMIT 10;",con)
no3

,mostWanted2003,quantityOrdered
0,1992 Ferrari 360 Spider red,672.0
1,1936 Mercedes-Benz 500K Special Roadster,429.0
2,1964 Mercedes Tour Bus,427.0
3,1940s Ford truck,408.0
4,1926 Ford Fire Engine,393.0
5,1956 Porsche 356A Coupe,389.0
6,1965 Aston Martin DB5,382.0
7,1948 Porsche Type 356 Roadster,382.0
8,1996 Peterbilt 379 Stake Bed with Outrigger,373.0
9,1950's Chicago Surface Lines Streetcar,372.0
